In [3]:
!pip install git+https://github.com/rwightman/pytorch-image-models
!pip install --upgrade wandb

  Cloning https://github.com/rwightman/pytorch-image-models to /tmp/pip-req-build-e6dfjjae
  Running command git clone -q https://github.com/rwightman/pytorch-image-models /tmp/pip-req-build-e6dfjjae
  Resolved https://github.com/rwightman/pytorch-image-models to commit 3f9959cdd28cb959980abf81fc4cf34f32e18399
  Created wheel for timm: filename=timm-0.4.13-py3-none-any.whl size=409519 sha256=554b986f16eb2cc4b720a2fc4d29c920f78826d5ebba1bb4d89ce9e436943188
  Stored in directory: /tmp/pip-ephem-wheel-cache-e0wtgcmy/wheels/69/3d/b0/be55cbadabd87a0e1875d63c7492d199097a39cc2433637650
Successfully built timm


In [4]:
import os
import gc
import cv2
import copy
import time
import random
from PIL import Image

# for data manipulation
import numpy as np
import pandas as pd

# pytorch import
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# sklearn import
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

# for descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING']='1'

Weights & Biases(W&B) is a set of machine learning tools that helps you build better models faster. kaggle competitions require fast-paced model development and evaluation. There are a lot components: exploring the training data, training diffent models, combining trained models in different combinations(ensembling), and so on.

W&B can be useful for Kaggle competition with it's lightweight and interoperable tools:

* Quickly track experiments,
* Version and iterate on datasets,
* Evaluate model performance,
* Reproduce models,
* Visualize results and spot regressions,
* Share findings with colleagues.

To learn more about Weights and Biases check out this kernel.

In [5]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret('wandb_api')
    wandb.login(key=api_key)
    anony=None
except:
    anony='must'
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


In [6]:
ROOT_DIR = "../input/petfinder-pawpularity-score"
TRAIN_DIR = "../input/petfinder-pawpularity-score/train"
TEST_DIR = "../input/petfinder-pawpularity-score/test"

In [7]:
# Traning Configure
CONFIG = dict(
seed=42,
model_name='tf_efficientnet_b4_ns',
train_batch_size=16,
valid_batch_size=32,
img_size=512,
epochs=5,
learning_rate=1e-4,
scheduler='CosineAnnealingLR',
min_lr=1e-6,
T_max=20,
T_0=25,
warmup_epochs=0,
weight_decay=1e-6,
n_accumulate=1,
n_fold=7,
num_classes=1,
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
competitions='PetFinder',
_wandb_kernel='deb')

In [8]:
# Set Seed for Reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark=False
    os.environ['PYTHONHASHSEED']=str(seed)

set_seed(CONFIG['seed'])

# Read Data

In [9]:
def get_train_file_path(id):
    return f'{TRAIN_DIR}/{id}.jpg'

In [13]:
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['file_path'] = df['Id'].apply(get_train_file_path)

feature_cols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'file_path']]

In [ ]:
# visualize images
#run = wandb.init(project='Pawpularity',
#                config=CONFIG,
#                job_type='Visualization',
#                anonymous='must')

In [ ]:
# preview_table = wandb.Table(columns=['Id', 'Image', 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'Pawpularity'])
# tmp_df = df.sample(1000, random_state=CONFIG['seed']).reset_index(drop=True)
# for i in tqdm(range(len(tmp_df))):
#     row = tmp_df.loc[i]
#     img=Image.open(row.file_path)
#     preview_table.add_data(row['Id'],
#                          wandb.Image(img),
#                          row['Subject Focus'],
#                          row['Eyes'],
#                          row['Face'],
#                          row['Near'],
#                          row['Action'],
#                          row['Accessory'],
#                          row['Group'],
#                          row['Collage'],
#                          row['Human'],
#                          row['Occlusion'],
#                          row['Info'],
#                          row['Blur'],
#                          row['Pawpularity'])
#wandb.log({'Visualization':preview_table})
#run.finish()

In [14]:
# Creat Folds
def create_folds(df, n_s=7, n_grp=None):
    df['kfold'] = -1
    if n_grp is None:
        skf = KFold(n_splits=n_s, random_state=CONFIG['seed'])
        target=df['Pawpularity']
    else:
        skf=StratifiedKFold(n_splits=n_s, shuffle=True, random_state=CONFIG['seed'])
        df['grp']=pd.cut(df['Pawpularity'], n_grp, labels=False)
        target = df.grp
    for fold_no, (t, v) in enumerate(skf.split(target, target)):
        df.loc[v, 'kfold'] = fold_no
    
    df = df.drop('grp', axis=1)
    
    return df

In [15]:
df = create_folds(df, n_s=CONFIG['n_fold'], n_grp=14)
df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,file_path,kfold
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...,1
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...,3
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...,1
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...,4
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...,5


In [16]:
# Dataset Class

class PawpularityDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir=root_dir  # 먼저 데이터 불러오기
        self.df = df
        self.file_names = df['file_path'].values
        self.targets = df['Pawpularity'].values
        self.meta = df[feature_cols].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        meta = self.meta[index, :]
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        
        return img, meta, target

In [17]:
# Augmentations

data_transforms = {
    'train' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225],
                   max_pixel_value=255.0,
                   p=1.0),
        ToTensorV2()
    ], p=1.0),
    
    'valid' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225],
                   max_pixel_value=255.0,
                   p=1.0),
        ToTensorV2()
    ], p=1.0)
}

# Create Model

In [92]:
class PawpularityModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(PawpularityModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.reset_classifier(0)
        self.fc = nn.Linear(self.n_features + 12, CONFIG['num_classes'])
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, images, meta):
        features = self.model(images)  # features = (bs, embedding_size)
        features = self.dropout(features)  
        features = torch.cat([features, meta], dim=1)  # features = (bs, embedding_size + 12)
        output = self.fc(features)  # outputs  = (bs, num_classes)
        return output

    
model = PawpularityModel(CONFIG['model_name'])
model.to(CONFIG['device'])

# 모델 설명 ↓↓

PawpularityModel(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, tra

In [30]:
# Loss Function
def criterion(outputs, targets):
    return torch.sqrt(nn.MSELoss()(outputs.view(-1), targets.view(-1)))

In [31]:
# Training Function

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, meta, targets) in bar:         
        images = images.to(device, dtype=torch.float)
        meta = meta.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            outputs = model(images, meta)
            loss = criterion(outputs, targets)
            loss = loss / CONFIG['n_accumulate']
            
        scaler.scale(loss).backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [32]:
# Validation function

@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size=0
    running_loss=0.00
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, meta, targets) in bar:
        images = images.to(device, dtype=torch.float)
        meta = meta.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        outputs = model(images, meta)
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        PREDS.append(outputs.view(-1).cpu().detach().numpy())
        TARGETS.append(targets.view(-1).cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
    
    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    val_rmse = mean_squared_error(TARGETS, PREDS, squared=False)
    
    gc.collect()
    
    return epoch_loss, val_rmse

In [33]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_rmse = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, val_epoch_rmse = valid_one_epoch(model, valid_loader, 
                                                         device=CONFIG['device'], 
                                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid RMSE'].append(val_epoch_rmse)
        
        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        wandb.log({"Valid RMSE": val_epoch_rmse})
        
        print(f'Valid RMSE: {val_epoch_rmse}')
        
        # deep copy the model
        if val_epoch_rmse <= best_epoch_rmse:
            print(f"{c_}Validation Loss Improved ({best_epoch_rmse} ---> {val_epoch_rmse})")
            best_epoch_rmse = val_epoch_rmse
            run.summary["Best RMSE"] = best_epoch_rmse
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "RMSE{:.4f}_epoch{:.0f}.bin".format(best_epoch_rmse, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            wandb.save(PATH)
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best RMSE: {:.4f}".format(best_epoch_rmse))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [34]:
def prepare_loader(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = PawpularityDataset(TRAIN_DIR, df_train, transforms=data_transforms['train'])
    valid_dataset = PawpularityDataset(TRAIN_DIR, df_valid, transforms=data_transforms['valid'])
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], num_workers=4, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], num_workers=4, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [35]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr'])
    
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=CONFIG['T_0'], eta_min=CONFIG['min_lr'])
    
    elif CONFIG['scheduler'] == None:
        return None
    
    return scheduler

In [36]:
# create Dataloader
train_loader, valid_loader = prepare_loader(fold=0)

In [37]:
# define optimizer and scheduler

optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

In [29]:
run = wandb.init(project='Pawpularity',
                config=CONFIG,
                job_type='Train',
                anonymous='must')

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
2021-09-29 06:20:58.724855: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



# Start Training

In [38]:
model, history = run_training(model, optimizer, scheduler, device=CONFIG['device'], num_epochs=CONFIG['epochs'])

[INFO] Using GPU: Tesla P100-PCIE-16GB



100%|██████████| 45/45 [00:33<00:00,  1.33it/s, Epoch=1, LR=4.28e-5, Valid_Loss=27]  


Valid RMSE: 27.366043090820312
Validation Loss Improved (inf ---> 27.366043090820312)
Model Saved



100%|██████████| 45/45 [00:33<00:00,  1.36it/s, Epoch=2, LR=3.42e-6, Valid_Loss=19.7]


Valid RMSE: 20.057537078857422
Validation Loss Improved (27.366043090820312 ---> 20.057537078857422)
Model Saved



...

Valid RMSE: 18.537817001342773
Validation Loss Improved (20.057537078857422 ---> 18.537817001342773)
Model Saved



100%|██████████| 45/45 [00:32<00:00,  1.37it/s, Epoch=4, LR=9.05e-5, Valid_Loss=20.2]


Valid RMSE: 20.511444091796875



100%|██████████| 45/45 [00:32<00:00,  1.36it/s, Epoch=5, LR=1.55e-5, Valid_Loss=19.9]


Valid RMSE: 20.20870590209961

Training complete in 0h 47m 21s
Best RMSE: 18.5378


In [39]:
run.finish()

Best RMSE,18.53782
Train Loss,11.80401
Valid Loss,19.89712
Valid RMSE,20.20871


Train Loss,█▄▃▂▁
Valid Loss,█▂▁▃▂
Valid RMSE,█▂▁▃▂


In [90]:
MODEL_PATHS = [
    #'../input/pawpularity-torch-rmse/RMSE18.5378_epoch3.bin',
    './RMSE18.5378_epoch3.bin'
    # '' # 모델 계속 돌리면서 최저 RMSE
]

In [73]:
CONFIG = dict(
seed = 42,
model_name = 'tf_efficientnet_b4_ns',
test_batch_size = 32,
img_size = 512,
num_classes = 1,
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

In [74]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [75]:
def get_test_file_path(id):
    return f'{TEST_DIR}/{id}.jpg'

df = pd.read_csv(f"{ROOT_DIR}/test.csv")
df['file_path'] = df['Id'].apply(get_test_file_path)

feature_cols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'file_path']]

In [76]:
class PawpularityTestDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.file_names = df['file_path'].values
        self.meta = df[feature_cols].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        meta = self.meta[index, :]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return img, meta

In [77]:
class PawpularityModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(PawpularityModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.reset_classifier(0)
        self.fc = nn.Linear(self.n_features + 12, CONFIG['num_classes'])
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, images, meta):
        features = self.model(images)                 # features = (bs, embedding_size)
        features = self.dropout(features)
        features = torch.cat([features, meta], dim=1) # features = (bs, embedding_size + 12)
        output = self.fc(features)                    # outputs  = (bs, num_classes)
        return output

In [93]:
# Augmentation
test_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225], 
            max_pixel_value=255.0, 
            p=1.0
        ),
    ToTensorV2()], p=1.)

In [94]:
# test model

@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, meta) in bar:        
        images = images.to(device, dtype=torch.float)
        meta = meta.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        dataset_size += batch_size
        
        outputs = model(images, meta)
        PREDS.append(outputs.view(-1).cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [99]:
def inference(model_paths,dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = PawpularityModel(CONFIG['model_name'], pretrained=False)
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [100]:
test_dataset = PawpularityTestDataset(TEST_DIR, df, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'], 
                         num_workers=4, shuffle=False, pin_memory=True)

In [101]:
preds = inference(MODEL_PATHS, test_loader, CONFIG['device'])

Getting predictions for model 1


100%|██████████| 1/1 [00:00<00:00,  3.09it/s]


In [88]:
sample = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sample['Pawpularity'] = preds

In [89]:
sample.to_csv('submit.csv', index=False)

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,2.714290
1,43a2262d7738e3d420d453815151079e,2.484810
2,4e429cead1848a298432a0acad014c9d,2.749248
3,80bc3ccafcc51b66303c2c263aa38486,2.580287
4,8f49844c382931444e68dffbe20228f4,2.783918
5,b03f7041962238a7c9d6537e22f9b017,2.741874
6,c978013571258ed6d4637f6e8cc9d6a3,2.755523
7,e0de453c1bffc20c22b072b34b54e50f,2.631648
